In [1]:
# !pip install tiktoken

In [9]:
import os
import json
import numpy as np
import pandas as pd
import tiktoken

base_dir = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/'


In [10]:
def read_data(path, allow_pickle=True):
    data = np.load(path, allow_pickle=allow_pickle)
    return data

In [14]:
train_data_dir_sp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/SP-train.npy'
train_data_dir_wp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/WP-train.npy'
val_data_dir_sp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/SP_eval_data_for_practice.npy'
val_data_dir_wp = '/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/WP_val_question_random.npy'

train_data_sp = read_data(train_data_dir_sp)
val_data_sp = read_data(val_data_dir_sp)

train_data_wp = read_data(train_data_dir_wp)
val_data_wp = read_data(val_data_dir_wp)

n_train_sp, n_train_select_Sp = train_data_sp.shape[0], 3
n_val_sp, n_val_select_sp = val_data_sp.shape[0], 3

n_train_wp, n_train_select_wp = train_data_wp.shape[0], 3
n_val_wp, n_val_select_Wp = val_data_wp.shape[0], 3

In [16]:
import numpy as np
import pandas as pd


# Convert the array of dictionaries to a pandas DataFrame
df_sp = pd.DataFrame(train_data_sp.tolist())
df_wp = pd.DataFrame(train_data_wp.tolist())
df2_sp = pd.DataFrame(val_data_sp.tolist())
df2_wp = pd.DataFrame(val_data_wp.tolist())




In [17]:
df_sp.to_csv(base_dir + "train_sp.csv")
df_wp.to_csv(base_dir + "train_wp.csv")
df2_sp.to_csv(base_dir + "test_sp.csv")
df2_wp.to_csv(base_dir + "test_wp.csv")


## Select static data

In [19]:

# target_file = base_dir + "sdata.json"

# if not os.path.exists(target_file):
#     with open(target_file, 'w') as f:
#         data = [train_data[0], train_data[n_train//2], train_data[n_train-1],
#                 val_data[0], val_data[n_val//2], val_data[n_val-1]]
#         f.write('[\n')
#         for d in data:
#             result = "{\n"
#             for k, v in d.items():
#                 result += f'\t"{k}": "{v}",\n'
#             result = result[0:-2] + "\n},\n"
#             f.write(result)
#         f.write(']')

## Convert train data to csv file

In [20]:
csv_train_file_sp = "/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_data_sp.csv"
csv_train_file_wp = "/home/drishya/drishya/drishya/NLP/SemEval-2024_task9_BRAINTEASER/train_data_wp.csv"
# columns = train_data[0].keys()
columns = ['id', 'question', 'answer', 'distractor1', 'distractor2', 'label']
train_df = pd.DataFrame(columns=columns)
for d in train_data_sp:
    train_df.loc[len(train_df)] = d
    
for d in train_data_wp:
    train_df.loc[len(train_df)] = d

In [21]:
print(train_df.head(1))

     id                                           question  \
0  SP-0  Mr. and Mrs. Mustard have six daughters and ea...   

                                   answer  \
0  Each daughter shares the same brother.   

                                         distractor1  \
0  Some daughters get married and have their own ...   

                                         distractor2  label  
0  Some brothers were not loved by family and mov...      1  


In [11]:
csv_val_file = base_dir + "val_data.csv"
# columns = train_data[0].keys()
columns = ['id', 'question', 'answer', 'distractor1', 'distractor2', 'label']
val_df = pd.DataFrame(columns=columns)
for d in val_data:
    val_df.loc[len(val_df)] = d

In [20]:
import numpy as np

# Load the .npy file
data = np.load(val_data_dir,allow_pickle=True)

# Check the contents of the array
# print(array_data)

# # To get more details about the shape and datatype
# print("Shape:", array_data.shape)
# print("DataType:", array_data.dtype)
#Convert data to a DataFrame
# Normalize the data to create a flat list of dictionaries, where each choice is a separate key
flat_data = []
for item in data:
    question = item['question']
    choices = item['choice_list']
    for idx, choice in enumerate(choices):
        # Create a new dictionary for each choice
        row = {
            'question': question,
            f'choice_{idx+1}': choice
        }
        flat_data.append(row)

# Convert the flat list of dictionaries to a DataFrame
df = pd.DataFrame(flat_data)

# Now, the DataFrame has a separate column for each choice
# Save the DataFrame to a CSV file
csv_path = base_dir + 'output_file.csv'
df.to_csv(csv_path, index=False)


In [14]:
enc = tiktoken.encoding_for_model("gpt-3.5-turbo-1106")
extra_columns = {'q_token_count': 'question', 'a_token_count': 'answer', 'd1_token_count': 'distractor1', 'd2_token_count': 'distractor2'}
for i in range(len(train_df)):
    for k, v in extra_columns.items():
        if isinstance(train_df.loc[i][v], str):
            train_df.loc[i, k] = len(enc.encode(train_df.loc[i][v]))

In [15]:
for i in range(len(train_df)):
    for k, v in extra_columns.items():
        if isinstance(train_df.loc[i][v], str):
            assert train_df.loc[i, k] == len(enc.encode(train_df.loc[i, v]))
train_df.to_csv(csv_train_file)
train_df.head()

,id,question,answer,distractor1,distractor2,label,q_token_count,a_token_count,d1_token_count,d2_token_count
0,SP-0,Mr. and Mrs. Mustard have six daughters and ea...,Each daughter shares the same brother.,Some daughters get married and have their own ...,Some brothers were not loved by family and mov...,1,33.0,7.0,10.0,11.0
1,SP-0_SR,The six daughters of Mr. and Mrs. Mustard each...,Each daughter shares the same brother.,Some daughters get married and have their own ...,Some brothers were not loved by family and mov...,2,31.0,7.0,10.0,11.0
2,SP-0_CR,"A chess team has five players, and each player...",Each player shares the same coach.,Some coaches get a raise.,Some players are backups and not allowed to play.,0,29.0,7.0,6.0,10.0
3,SP-1,A woman shoots her husband. Then she holds him...,The woman was a photographer. She shot a pictu...,The woman gets arrested for murder after dinner.,The woman gets a new partner.,2,45.0,24.0,9.0,7.0
4,SP-1_SR,An individual shoots their spouse. She continu...,The woman was a photographer. She shot a pictu...,The woman gets arrested for murder after dinner.,The woman gets a new partner.,1,43.0,24.0,9.0,7.0


In [16]:
train_df['question'][1]

'The six daughters of Mr. and Mrs. Mustard each have one brother. However, the family only consists of nine people; how is that possible?'

In [17]:
total_token_count = 0
for k, v in extra_columns.items():
    total_token_count += train_df[k].sum()
    print(f'sum {v} token count: {str(train_df[k].sum())}')
print('----------------------------')
print('the number of the total tokens: ' + str(total_token_count))

sum question token count: 21268.0
sum answer token count: 5702.0
sum distractor1 token count: 5403.0
sum distractor2 token count: 5446.0
----------------------------
the number of the total tokens: 37819.0
